# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### Tecnológico de Monterrey

### Prof Dra. Grettel Barceló Alonso y Dr. Luis Eduardo Falcón Morales

## Avance III de Proyecto

## Modelo Base

## Integrantes del Equipo:
### - Erika Cardona Rojas            A01749170
### - Miriam Bönsch                  A01330346
### - Mardonio Manuel Román Ramírez  A01795265

In [7]:
import time
import logging
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut, RepeatedKFold, RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Librerias de modelos
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

import yaml

# Cargando Yaml
with open("../config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

In [2]:
# Cargando Base de Datos
df = pd.read_excel(r"../Data/DF_Pred_4.xlsx")

# Filtrado De Variables

Dada nuestra previa entrega, se utilizarán las variables más importantes en la RFECV

In [3]:
# Cargando las variables más importantes según RFECV 1
df_ranking = pd.read_excel('../Entregables/UdeBarcelona/RFECV.xlsx')

selected_features_1 = df_ranking.loc[df_ranking['Ranking'] == 1, 'Feature']

# Cargando las variables más importantes según RFECV 2
df_ranking = pd.read_excel('../Entregables/UdeBarcelona/RFECV_Lasso.xlsx')

selected_features_2 = df_ranking.loc[df_ranking['Ranking'] == 1, 'Feature']

del df_ranking

In [4]:
# Obteniendo variables únicas de ambos
unique_selected_features = list(set(pd.concat([selected_features_1, selected_features_2])))

# Entrenamiento de Modelos

## Justificación de la Estrategia de Validación y Optimización de Modelos

Para la fase de modelado predictivo de este estudio, se ha implementado un flujo de trabajo de Machine Learning enfocado en maximizar la confiabilidad de las métricas de evaluación, mitigando los riesgos inherentes al trabajo con muestras de tamaño reducido. El código anterior se fundamenta en los siguientes pilares metodológicos:

### 1. Validación Cruzada Robusta (`RepeatedKFold`)

Dado el tamaño de la muestra, un K-Fold tradicional podría presentar una alta varianza en la estimación del error dependiendo de cómo se realicen las particiones aleatorias. En su lugar, se optó por una estrategia de **K-Fold Repetido** (`n_splits=7`, `n_repeats=5`). 
* Al usar 7 particiones, aseguramos que el conjunto de validación de cada pliegue contenga un número razonable de observaciones (aproximadamente 6), permitiendo que el conjunto de entrenamiento mantenga la mayor parte de la varianza original.
* Repetir el proceso 5 veces con semillas aleatorias diferentes estabiliza las métricas de evaluación (RMSE, MAE, R2), promediando el rendimiento sobre 35 escenarios de validación distintos. Esto ofrece una estimación del error de generalización mucho más sólida y menos susceptible al azar que un K-Fold simple o un Leave-One-Out (el cual puede sufrir de alta varianza frente a valores atípicos).

### 2. Prevención Estricta de Fuga de Datos (Data Leakage)
Para garantizar la validez de los resultados, es imperativo que las transformaciones aplicadas a los datos no filtren información del conjunto de validación al conjunto de entrenamiento. Para ello, se encapsuló el `StandardScaler` dentro de un objeto `Pipeline` de Scikit-Learn. De esta manera, durante cada una de las 35 iteraciones de la validación cruzada, el escalador se ajusta (*fit*) estrictamente sobre los pliegues de entrenamiento, y solo se utiliza para transformar el pliegue de validación. 

### 3. Optimización Estocástica de Hiperparámetros (`RandomizedSearchCV`)
En lugar de una búsqueda de cuadrícula exhaustiva (`GridSearchCV`), se implementó `RandomizedSearchCV`. Esta técnica evalúa una muestra aleatoria de combinaciones del espacio de hiperparámetros (limitado a `n_iter=40` o al máximo de combinaciones posibles). La literatura demuestra que la búsqueda aleatoria es computacionalmente más eficiente e igual de efectiva que la búsqueda en cuadrícula para encontrar modelos óptimos, evitando el desgaste de recursos en zonas del espacio de búsqueda con bajo impacto en la función de pérdida.

### 4. Arquitectura Reproducible y Escalable
El código se diseñó utilizando un patrón **Model Registry**, separando la definición lógica de los algoritmos de su ejecución. Junto con la integración de la librería `logging`, esta estructura asegura que los experimentos sean totalmente trazables, reproducibles y fácilmente extensibles en caso de que se requiera evaluar nuevos modelos o distribuciones de parámetros en futuras etapas de la investigación.

> ### Creación de DF X e Y

In [5]:
X_full = df[unique_selected_features]
y = df.loc[:,'delta_bdnf_Int']

# Separación en Train y Test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)

> ### ENTRENAMIENTO CON TODAS LAS VARIABLES IMPORTANTES
> #### SEGUN AMBAS TECNICAS DE RFECV

In [8]:
# Registry de modelos
MODEL_REGISTRY = {
    "Ridge": Ridge,
    "Lasso": Lasso,
    "ElasticNet": ElasticNet,
    "SVR": SVR,
    "KNeighborsRegressor": KNeighborsRegressor,
    "DecisionTreeRegressor": DecisionTreeRegressor,
    "MLPRegressor": MLPRegressor,
}

cv_strategy = RepeatedKFold(
    n_splits=7,
    n_repeats=5,
    random_state=42
)

# Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

results = []
best_estimators = {}

for name, model_config in config['models_and_params'].items():

    start_time = time.time()
    logger.info(f"Optimizando modelo: {name}")

    # -------- Extraer configuración ----------
    model_class_name = model_config['model']['class']
    model_params = model_config['model']['params']
    param_grid = model_config['param_grid']

    model = MODEL_REGISTRY[model_class_name](**model_params)

    # -------- Pipeline  ----------
    pipeline = Pipeline(
        steps=[
            ('scaler', StandardScaler()),
            ('model', model)
        ],
        memory=None 
    )

    # -------- RandomizedSearch ----------
    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_grid,
        n_iter=min(40, np.prod([len(v) for v in param_grid.values()])),
        cv=cv_strategy,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )

    search.fit(X_train, y_train)

    best_pipeline = search.best_estimator_
    best_estimators[name] = best_pipeline

    # -------- Evaluación en test ----------
    y_pred = best_pipeline.predict(X_test)

    rmse = root_mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    clean_params = {
        k.replace('model__', ''): v
        for k, v in search.best_params_.items()
    }

    elapsed = time.time() - start_time

    results.append({
        'Modelo': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Mejores Hiperparámetros': clean_params,
        'Tiempo (s)': round(elapsed, 2)
    })

    logger.info(f"{name} terminado en {elapsed:.2f} segundos")

INFO:__main__:Optimizando modelo: Ridge
INFO:__main__:Ridge terminado en 2.59 segundos
INFO:__main__:Optimizando modelo: Lasso
INFO:__main__:Lasso terminado en 0.26 segundos
INFO:__main__:Optimizando modelo: ElasticNet
INFO:__main__:ElasticNet terminado en 0.58 segundos
INFO:__main__:Optimizando modelo: SVR
INFO:__main__:SVR terminado en 0.46 segundos
INFO:__main__:Optimizando modelo: KNN Regressor
INFO:__main__:KNN Regressor terminado en 1.77 segundos
INFO:__main__:Optimizando modelo: Decision Tree
INFO:__main__:Decision Tree terminado en 0.58 segundos
INFO:__main__:Optimizando modelo: MLP Regressor
INFO:__main__:MLP Regressor terminado en 45.87 segundos


In [ ]:
# Generando DataFrame Ordenado
results_df_all = (
    pd.DataFrame(results)
      .sort_values("RMSE")
      .reset_index(drop=True)
)

results_df_all

,Modelo,RMSE,MAE,R2,Mejores Hiperparámetros,Tiempo (s)
0,Lasso,36.375093,23.362040,0.571879,"{'selection': 'cyclic', 'alpha': 0.01}",0.26
1,SVR,45.624454,26.750614,0.326475,"{'kernel': 'linear', 'gamma': 'auto', 'epsilon...",0.46
2,KNN Regressor,52.128744,30.978001,0.120749,"{'weights': 'distance', 'p': 1, 'n_neighbors': 5}",1.77
3,ElasticNet,53.454563,28.582199,0.075455,"{'l1_ratio': 0.3, 'alpha': 1.0}",0.58
4,Ridge,56.820228,29.889415,-0.044634,"{'solver': 'sag', 'alpha': 100.0}",2.59
5,MLP Regressor,58.441065,33.918513,-0.105082,"{'solver': 'lbfgs', 'hidden_layer_sizes': [100...",45.87
6,Decision Tree,78.411801,46.707459,-0.989396,"{'min_samples_split': 20, 'min_samples_leaf': ...",0.58


> ### ENTRENAMIENTO CON TODAS LAS VARIABLES IMPORTANTES
> #### SEGUN TECNICAS DE RFECV CON LASSO

> ### ENTRENAMIENTO CON TODAS LAS VARIABLES IMPORTANTES
> #### SEGUN TECNICAS DE RFECV CON XG-BOOST